# SDF Model Example 1: Using Pre-trained Models

In this jupyter notebook, we're going to showcase how to use and deploy several of the pre-trained models we have developed using this data.

Data and datasets are found in the `Data/` subfolder. Helper scripts and the `SDFModel` class are available in the `Scripts/` subfolder.

In [1]:
# enables direct importing of .py files
import sys
sys.path.insert(0, "Scripts/")

#### Version Control

We output the version of all the main software to run this example. Optional software is included, which can be removed from the set if needed.

In [2]:
import version
version.control({"numpy","scipy","matplotlib","pandas","sklearn",
                 "joblib","tqdm","jupyter","xgboost","seaborn","xlrd","pip"})

python: 3.9.4 | packaged by conda-forge | (default, May 10 2021, 22:10:34) [MSC v.1916 64 bit (AMD64)]
pip: 21.1.2
scipy: 1.6.2
numpy: 1.19.2
xlrd: 2.0.1
tqdm: 4.59.0
jupyter has no __version__ attribute.
pandas: 1.2.3
matplotlib: 3.3.4
xgboost: 1.4.0
joblib: 1.0.1
seaborn: 0.11.1
sklearn: 0.24.1


### Other imports

In [3]:
%matplotlib inline
import numpy as np
import joblib

from sdfmodel import SDFModel
from pandawrapper import read

## Unzipping data file

If you haven't done so already, you will want to unzip the `datasets.zip` file. This can be done manually or by executing the following cell:

In [4]:
#import zipfile
#with zipfile.ZipFile("Data/datasets.zip", 'r') as zip_ref:
#    zip_ref.extractall("Data/")

## Loading Pre-Trained Models

This work follows from the thesis, whereby not only do we develop the curated dataset(s) which can be used, but we provide some basic pre-trained models which can plug-and-play given gene, mRNA or protein labels, the corresponding estimated protein abundance. Here are the following pre-trained models in the `Models/` subfolder. Read the `Develop_Pretrained_Models` iPython notebook to see the code for how these models were made.

* `xgb_basic.pkl`: This pre-trained model was trained on `sPCA_dg_mca.csv` sequence-derived features and PAXDB protein abundance as found in `PAXDB_WHOLE_CONSENSUS.csv` as attached. 

Let's introduce the `SDFModel` class, a wrapper class for holding lots of data and having an extremely simple interface for accessing protein predictions.

### Example 1: Random Uniprot IDs

Here we will load in our sequence-derived feature dataset, randomly sample a few Uniprot IDs, and use these labels to generate predicted protein abundances:

In [19]:
# loading in sdfs
sdf_set = read("Data/sPCA_dg_mca.csv")
# randomly sample 5 IDs
choice_rng = np.random.choice(sdf_set['uniprot_id'].dropna().unique(), size=5)

# build model and get predictions
with SDFModel() as wrapper:
    pred1 = wrapper.predict(choice_rng)
    
# also acceptable is

# model = SDFModel()
# model.load()

#### Under the hood:

What's happening here is as follows:

1. We create a context using `with`, which loads in a series of files including HGNC label data, SDF data and the pre-trained model (in this case an [XGBoost](https://xgboost.ai/) regressor).
2. We call `predict()` passing in some Uniprot IDs. These IDs are compared against every potential label column within HGNC, and the one with the largest hits is kept. Duplicates for this column are dropped and we select rows within the SDF dataset that map to the IDs. This subset is then mildly preprocessed and passed into the pre-trained model's `predict()` function yielding estimates of protein abundance.

Here we see the predictions for a selection of random proteins. 

In [20]:
pred1

A0PJE2    19.080572
Q8TEK3    20.743275
Q5T292    17.719656
P09093    20.052114
H3BTG2    17.837173
dtype: float32

What is returned is a `pandas.Series` object which contains two parts:

- An Index object: i.e the Uniprot ID, accessible via the `.index` attribute.
- A values object: i.e the numbers, as a `numpy.ndarray`, accessible via the `.values`. attribute.

In [21]:
pred1.index

Index(['A0PJE2', 'Q8TEK3', 'Q5T292', 'P09093', 'H3BTG2'], dtype='object')

In [22]:
pred1.values

array([19.080572, 20.743275, 17.719656, 20.052114, 17.837173],
      dtype=float32)

### Example 2: Proteins associated with Histones

Something much more powerful is the in-built regular expression engine which we can utilize to match a [regular expression](https://en.wikipedia.org/wiki/Regular_expression) with HGNC, Uniprot, Ensembl and more. To activate this, set `regex=True` within the `predict()` method, passing in an appropriate regular expression string:

In [23]:
ypred_hist = wrapper.predict("HIST", regex=True)

C:\Users\Gregory\Documents\Git\sdfmodel\sdfmodel.py:141: UserWarning: merging operation returns 23 out of 54 selections.
  warnings.warn(f"merging operation returns {_tot} out of {N} selections.", UserWarning)


In [24]:
ypred_hist

HIST2H2BF     29.221022
HIST4H4       28.984224
HIST1H2BJ     29.531399
HIST1H1A      27.058922
HIST1H1D      28.816559
HIST2H2BE     29.361803
HIST1H2AA     28.377468
HIST1H2BL     29.116718
HIST1H1B      29.263620
HIST1H1C      29.342360
HIST1H1T      26.466911
HIST1H2AC     28.521139
HIST1H1E      29.247229
HIST1H2BD     29.470533
HIST3H2A      28.879248
HIST2H3C      28.640995
HIST2H2AA4    28.388741
HIST2H3A      28.640995
HIST1H2AG     28.798422
HIST1H2BK     29.773870
HIST1H2AI     28.780424
HIST1H2AD     28.532949
HIST3H2BB     29.253187
dtype: float32

Note that HGNC contains have over 50 histone-associated proteins, there were only 23 that uniquely mapped to the SDF dataset via Refseq or Uniprot IDs. Hence it is preferable where possible to associate your data using Refseq or Uniprot IDs to maximise the chance of a successful merge. 

## Delving deeper

If you type `SDFModel?` and run the box, you will see a pop up giving the documentation of the function. You may notice three key parameters involved:

1. `model`: A pre-trained model such as XGBoost regressor
2. `hgnc_data`: A string or `pandas.DataFrame` holding information from HGNC regarding Uniprot/Ensembl/Refseq labels.
3. `sdf_data`: The sequence-derived feature dataset, in any form.

Note that `SDFModel` does not perform any preprocessing on SDFs or the model, so if you want custom behaviour you will need to implement this yourself.

Look at the example 2 notebook for details of building the model yourself from scratch, or incorporating your own expression or function data.

## Extracting sequence information

We also allow users to retrieve the sequence-information used for the prediction using the `get_X()` function, passing the same sort of arguments as `predict()`:

In [25]:
sdf_hist = wrapper.get_X("HIST", regex=True)

This returns a `pandas.DataFrame` where the columns contain the preprocessed sequence-derived features. The `PC_` prefix refers to the PCA transformation performed on this feature block (for instance mononucleotide-frequencies or 'mononuc_freq' for short). MCA features are amino-acid meta features compressed into a reduced space.

Further details on the sequence-derived features can be found by looking at the `wrapper.SDF` object, which is a `PandaWrapper` custom object (`pandas.DataFrame` extended) with all the SDF information.

We can examine the first 5 elements using the `.head()` function:

In [26]:
sdf_hist.head()

,PC_mononuc_freq1,PC_mononuc_freq2,PC_mononuc_freq3,PC_mononuc_freq4,PC_mononuc_freq5,PC_mononuc_freq6,PC_mononuc_freq7,PC_mononuc_freq8,PC_length1,PC_length2,...,MCA15,MCA16,MCA17,MCA18,MCA19,MCA20,MCA21,MCA22,MCA23,MCA24
HIST2H2BF,1.551194,4.234976,0.882136,0.771362,1.560955,0.762836,1.256641,-2.266010,4.020590,1.091487,...,0.226129,0.005118,-0.034526,-0.140586,0.094006,-0.017614,0.232691,0.010540,-0.037639,-0.116309
HIST4H4,3.097717,1.959984,-0.250161,-0.917209,0.923793,0.797303,0.159189,-2.256613,4.512522,1.049775,...,0.145326,-0.087231,0.124367,0.067893,0.005953,-0.032204,0.129857,-0.045251,-0.029975,-0.073027
HIST1H2BJ,1.212988,2.723823,0.297347,1.093486,1.535689,1.098086,1.140610,-2.164952,4.111015,1.229294,...,0.128936,0.008844,0.044085,-0.088423,0.049840,0.002773,0.108543,-0.022683,-0.029753,-0.088405
HIST1H1A,-0.546653,1.853111,0.710686,0.382463,2.294800,1.739040,0.554972,-2.576994,2.874803,1.364153,...,0.107329,0.042031,0.018070,-0.084658,0.016924,-0.026044,0.085044,0.057265,-0.089144,0.087073
HIST1H1D,0.104160,3.506612,0.468844,-0.868451,2.493956,0.877680,-0.728570,-1.524019,2.904579,1.587633,...,0.173593,0.054218,0.043127,-0.120866,0.027369,-0.019529,0.080314,0.034861,-0.171507,0.077141


### Main objects

The `SDFModel` class has three major objects which users can directly access:

1. `model`: This is the pre-trained model. Note that for `predict()` to work, the model object must have a `predict` function, and must be fitted with valid coefficients. In this example, we used the `xgboost.XGBRegressor` class to fit our data to.
2. `hgnc`: This is a PandaWrapper containing labels. It doesn't necessarily have to be from HGNC, but they have a formal gene naming convention which makes it a natural table to align different foreign database labels to.
3. `SDF`: sequence-derived feature dataset that is preprocessed.

In [29]:
wrapper.SDF

PandaWrapper(sPCA_dg_mca(n=15269, p=137, mem=16.735MB))

In [30]:
wrapper.hgnc

PandaWrapper(hgnc_labels(n=504934, p=8, mem=32.316MB))

In [31]:
wrapper.model

XGBRegressor(alpha=0.00016833956095964933, base_score=0.5, booster='gbtree',
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             gamma=0, gpu_id=-1, importance_type='gain',
             interaction_constraints='', lambda=0.0022679853779692554,
             learning_rate=0.07466704456230339, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0.000168339568, reg_lambda=0.00226798537,
             scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)


You could then simply generate the predictions yourself using the pre-trained model provided:

In [17]:
wrapper.model.predict(sdf_hist)

array([29.221022, 28.984224, 29.531399, 27.058922, 28.816559, 29.361803,
       28.377468, 29.116718, 29.26362 , 29.34236 , 26.466911, 28.52114 ,
       29.247229, 29.470533, 28.879248, 28.640995, 28.38874 , 28.640995,
       28.798422, 29.77387 , 28.780424, 28.53295 , 29.253187],
      dtype=float32)